$\textbf{x}\in\Omega$

$$\frac{\partial u}{\partial t}+\textbf{a}\cdot\nabla u = \nabla\cdot(\mathsf{D}\cdot\nabla u) + r(\textbf{x})u + s(\textbf{x})$$

$$\int_\Omega\text{d}\Omega~v\frac{\partial u}{\partial t}+v\,\textbf{a}\cdot\nabla u + \nabla v\cdot(\mathsf{D}\cdot\nabla u) - vru - vs - \int_{\partial\Omega}\text{d}s~v\varepsilon\,\textbf{n}\cdot\nabla u=0 \qquad \forall v\in V$$

SUPG stabilization

$$\tau = \dots$$

In [ ]:
import numpy as np
from ufl.core.expr import Expr
from ufl import SpatialCoordinate, as_vector, cos, sqrt
from lucifex.mesh import rectangle_mesh
from lucifex.fem import Function, Constant
from lucifex.fdm import FunctionSeries, ConstantSeries, FiniteDifference
from lucifex.solver import ibvp, BoundaryConditions, evaluation
from lucifex.sim import Simulation, run
from lucifex.viz import plot_line
from lucifex.io import write, get_ipynb_file_name
from lucifex.pde.advection_diffusion import advection_diffusion_reaction


def source(
    t: Constant | float,
    j_xy: Expr,
) -> Expr:
    return np.exp(-float(t)**10) * j_xy


def create_simulation(
    supg: str | None,
    fdm: FiniteDifference,
    Lx: float,
    Ly: float,
    Nx: int,
    Ny: int,
    d: float, 
    dt: float,
) -> Simulation:
    order = max(fdm.order, 1)
    mesh = rectangle_mesh(Lx, Ly, Nx, Ny)
    x = SpatialCoordinate(mesh)
    a = as_vector((-x[1], x[0])) 
    d = Constant(mesh, d, 'd')
    dt = Constant(mesh, dt, name='dt')
    j_xy = cos(0.5 * np.pi * sqrt(x[0]**2 + x[1]**2))

    t = ConstantSeries(mesh, name='t', ics=0.0)
    u = FunctionSeries((mesh, 'P', 1), name='u', order=order, store=1)
    j = FunctionSeries((mesh, 'P', 1), name='j', order=order, store=1)

    s_solver = evaluation(j, source)(t[0], j_xy)

    bcs = BoundaryConditions(
        ('dirichlet', lambda x: x[0], 0.0),
        ('dirichlet', lambda x: x[1], 0.0),
        ('dirichlet', lambda x: x[0] - Lx, 0.0),
        ('dirichlet', lambda x: x[0] - Ly, 0.0),
    )
    a_solver = ibvp(advection_diffusion_reaction, ics=0.0, bcs=bcs)(
        u, dt, a, d, j=j, supg=supg,
    )

    solvers = [s_solver, a_solver]

    return Simulation(solvers, t, dt)



simulation = create_simulation()

n_stop = 50
run(simulation, n_stop)